<a href="https://colab.research.google.com/github/muhammadghazyy/Kaggle-Stuffs-2021/blob/main/Natural_Language_Processing_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from google.colab import files

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"muhammadghazy","key":"fd1a8a8edd66f6104869b0233c0a8724"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c nlp-getting-started

  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 65.5MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 137MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 53.1MB/s]


In [5]:
!ls

kaggle.json  sample_data  sample_submission.csv  test.csv  train.csv


In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [11]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [9]:
test.isna().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

# Text only NLP Model (without keyword and location)



In [10]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [14]:
train_x = np.array(train['text'])
train_y = np.array(train['target'])

In [15]:
train_x

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [37]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_x)

In [38]:
train_sequence = tokenizer.texts_to_sequences(train_x)
padded_train = tf.keras.preprocessing.sequence.pad_sequences(train_sequence, maxlen = max_length, truncating= trunc_type, padding= padding_type)

In [30]:
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics = ['acc']
    )

In [67]:
model.fit(padded_train, train_y, epochs = 25)

Epoch 1/25
238/238 [==============================] - 18s 75ms/step - loss: 0.3633 - acc: 0.8522
Epoch 2/25
238/238 [==============================] - 18s 75ms/step - loss: 0.3540 - acc: 0.8600
Epoch 3/25
238/238 [==============================] - 18s 75ms/step - loss: 0.3499 - acc: 0.8585
Epoch 4/25
238/238 [==============================] - 18s 76ms/step - loss: 0.3377 - acc: 0.8704
Epoch 5/25
238/238 [==============================] - 18s 76ms/step - loss: 0.3236 - acc: 0.8782
Epoch 6/25
238/238 [==============================] - 18s 75ms/step - loss: 0.3260 - acc: 0.8764
Epoch 7/25
238/238 [==============================] - 18s 75ms/step - loss: 0.3077 - acc: 0.8889
Epoch 8/25
238/238 [==============================] - 18s 76ms/step - loss: 0.2999 - acc: 0.8926
Epoch 9/25
238/238 [==============================] - 18s 76ms/step - loss: 0.2894 - acc: 0.8965
Epoch 10/25
238/238 [==============================] - 18s 74ms/step - loss: 0.2816 - acc: 0.9010
Epoch 11/25
238/238 [========

In [68]:
test_sequence = tokenizer.texts_to_sequences(np.array(test['text']))
padded_test = tf.keras.preprocessing.sequence.pad_sequences(test_sequence, maxlen = max_length, truncating= trunc_type, padding= padding_type)

In [69]:
res = model.predict(padded_test)

In [70]:
res1 = []
for r in res:
  if r >=0.5:
    res1.append(1)
  else:
    res1.append(0)

In [71]:
res1 = np.array(res1)

In [72]:
test1 = test.drop(['keyword','location','text'],axis=1)

In [73]:
test1['target'] = res1

In [74]:
test1.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [75]:
test1.to_csv('submission.csv',index = False)
!kaggle competitions submit -c nlp-getting-started -f submission.csv -m "Message"

100% 22.2k/22.2k [00:00<00:00, 106kB/s]
Successfully submitted to Natural Language Processing with Disaster Tweets

# Adding keywords and location